In [14]:
######################################## Import Statements ######################################
import pandas as pd
import holidays

In [15]:
############################### LOAD CTA DATA INTO DF ################################
df = pd.read_csv("CTA_Ridership_L_Station_Entries_Daily_Totals_20241112.csv")

# Convert to datetime format
df['date'] = pd.to_datetime(df['date'])

# Sort the df in ascending order of the date column & only include date ranges we want
df_sorted = df.sort_values(by='date')
df_sorted = df_sorted[(df_sorted['date'] >= '2017-01-01') & (df_sorted['date'] <= '2024-08-31')]

############################### CREATE A DF FOR US HOLIDAYS DATA ################################
us_holidays = holidays.US(years=range(2017, 2025))  # Set desired year range
holiday_df = pd.DataFrame({
    "date": list(us_holidays.keys()),
    "holiday_name": list(us_holidays.values())
})
# Convert the 'date' column to datetime
holiday_df['date'] = pd.to_datetime(holiday_df['date'])

# Sort df by 'date' 
sorted_holiday_df = holiday_df.sort_values(by='date')
filtered_holiday_df = sorted_holiday_df[(sorted_holiday_df['date'] >= '2017-01-01') & (sorted_holiday_df['date'] <= '2024-08-31')]

print(filtered_holiday_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 84 entries, 0 to 81
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          84 non-null     datetime64[ns]
 1   holiday_name  84 non-null     object        
dtypes: datetime64[ns](1), object(1)
memory usage: 2.0+ KB
None


In [16]:
############################### FEATURE ENGINEERING, CREATE A BINARY COL NAMED is_holiday ################################
df_sorted['is_holiday'] = df_sorted['date'].isin(filtered_holiday_df['date']).astype(int)

###################################### MERGING HOLIDAYS DF WITH CTA DATAFRAME USING DATE ################################
merged_data = pd.merge(
    df_sorted, 
    filtered_holiday_df, 
    left_on='date', 
    right_on='date', 
    how='left'
)
#Create new columns for Month, Day, Year, and Day of the Week in the merged df
merged_data['Year'] = merged_data['date'].dt.year
merged_data['Month'] = merged_data['date'].dt.month
merged_data['Day'] = merged_data['date'].dt.day
merged_data['Weekday'] = merged_data['date'].dt.day_name()
merged_data['around_holiday']=0

########## FEATURE ENGINEERING, CREATE A BINARY COL 'around_holidays'=1 for the 2 days before a holiday######################
holiday_dates = merged_data[merged_data['is_holiday'] == 1]['date']
for holiday in holiday_dates:
    merged_data.loc[(merged_data['date'] >= holiday - pd.Timedelta(days=2)) & (merged_data['date'] < holiday), 'around_holiday'] = 1

In [17]:
merged_data

,station_id,stationname,date,daytype,rides,is_holiday,holiday_name,Year,Month,Day,Weekday,around_holiday
0,41380,Bryn Mawr,2017-01-01,U,2083,1,New Year's Day,2017,1,1,Sunday,1
1,40480,Cicero-Lake,2017-01-01,U,533,1,New Year's Day,2017,1,1,Sunday,1
2,41050,Linden,2017-01-01,U,225,1,New Year's Day,2017,1,1,Sunday,1
3,41210,Wellington,2017-01-01,U,835,1,New Year's Day,2017,1,1,Sunday,1
4,40450,95th/Dan Ryan,2017-01-01,U,5102,1,New Year's Day,2017,1,1,Sunday,1
...,...,...,...,...,...,...,...,...,...,...,...,...
401372,41150,Kedzie-Midway,2024-08-31,A,1433,0,NaN,2024,8,31,Saturday,0
401373,41140,King Drive,2024-08-31,A,210,0,NaN,2024,8,31,Saturday,0
401374,40170,Ashland-Lake,2024-08-31,A,2901,0,NaN,2024,8,31,Saturday,0
401375,41180,Kedzie-Brown,2024-08-31,A,619,0,NaN,2024,8,31,Saturday,0


In [18]:
merged_data.to_csv("cta_merged.csv", index=False)